In [75]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Carga de datos

In [76]:
data = pd.read_csv('Maternal Health Risk Data Set.csv')
data

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk
1010,55,120,90,18.0,98.0,60,high risk
1011,35,85,60,19.0,98.0,86,high risk
1012,43,120,90,18.0,98.0,70,high risk


In [77]:
data['RiskLevel'] = data['RiskLevel'].map({'low risk': 0, 'mid risk': 1, 'high risk': 2})
data['PulsePressure'] = data['SystolicBP'] - data['DiastolicBP']
data['MAP'] = data['DiastolicBP'] + (data['PulsePressure'] / 3)
data['BPRatio'] = data['SystolicBP'] / data['DiastolicBP']
data['SBP/Age'] = data['SystolicBP'] / data['Age']
data['DBP/Age'] = data['DiastolicBP'] / data['Age']
data['BS/Age'] = data['BS'] / data['Age']
data.drop(columns=(['HeartRate', 'DiastolicBP']), inplace=True)
data

,Age,SystolicBP,BS,BodyTemp,RiskLevel,PulsePressure,MAP,BPRatio,SBP/Age,DBP/Age,BS/Age
0,25,130,15.0,98.0,2,50,96.666667,1.625000,5.200000,3.200000,0.600000
1,35,140,13.0,98.0,2,50,106.666667,1.555556,4.000000,2.571429,0.371429
2,29,90,8.0,100.0,2,20,76.666667,1.285714,3.103448,2.413793,0.275862
3,30,140,7.0,98.0,2,55,103.333333,1.647059,4.666667,2.833333,0.233333
4,35,120,6.1,98.0,0,60,80.000000,2.000000,3.428571,1.714286,0.174286
...,...,...,...,...,...,...,...,...,...,...,...
1009,22,120,15.0,98.0,2,60,80.000000,2.000000,5.454545,2.727273,0.681818
1010,55,120,18.0,98.0,2,30,100.000000,1.333333,2.181818,1.636364,0.327273
1011,35,85,19.0,98.0,2,25,68.333333,1.416667,2.428571,1.714286,0.542857
1012,43,120,18.0,98.0,2,30,100.000000,1.333333,2.790698,2.093023,0.418605


In [78]:
x = data.drop(columns=['RiskLevel'])
y = data['RiskLevel']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [79]:
classes = np.unique(y_train)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights = dict(zip(classes, class_weights_array))
print("Class Weights:", class_weights)

model = tf.keras.Sequential([

    # Layer 1
    tf.keras.layers.Dense(
        64, activation='relu',
        input_shape=(x_train.shape[1],),
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    # Layer 2
    tf.keras.layers.Dense(
        32, activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    # Layer 3
    tf.keras.layers.Dense(
        16, activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    ),

    # Output layer
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=12,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(x_test, y_test),
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=2
)

y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

y_pred_train_classes = np.argmax(y_pred_train, axis=1)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)


Class Weights: {np.int64(0): np.float64(0.8343023255813954), np.int64(1): np.float64(1.0141342756183747), np.int64(2): np.float64(1.2264957264957266)}
Epoch 1/200


/Users/sofia/Desktop/séptimo/ProyectoFinalMIA/venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 - 1s - 29ms/step - accuracy: 0.4077 - loss: 1.3527 - val_accuracy: 0.4771 - val_loss: 1.1384
Epoch 2/200
27/27 - 0s - 2ms/step - accuracy: 0.5064 - loss: 1.1628 - val_accuracy: 0.5948 - val_loss: 1.0862
Epoch 3/200
27/27 - 0s - 1ms/step - accuracy: 0.5238 - loss: 1.0835 - val_accuracy: 0.6144 - val_loss: 1.0495
Epoch 4/200
27/27 - 0s - 1ms/step - accuracy: 0.5494 - loss: 1.0185 - val_accuracy: 0.6667 - val_loss: 1.0162
Epoch 5/200
27/27 - 0s - 1ms/step - accuracy: 0.5947 - loss: 0.9873 - val_accuracy: 0.6732 - val_loss: 0.9894
Epoch 6/200
27/27 - 0s - 1ms/step - accuracy: 0.6039 - loss: 0.9586 - val_accuracy: 0.6863 - val_loss: 0.9654
Epoch 7/200
27/27 - 0s - 1ms/step - accuracy: 0.5900 - loss: 0.9490 - val_accuracy: 0.6928 - val_loss: 0.9298
Epoch 8/200
27/27 - 0s - 1ms/step - accuracy: 0.6190 - loss: 0.9076 - val_accuracy: 0.7190 - val_loss: 0.9001
Epoch 9/200
27/27 - 0s - 1ms/step - accuracy: 0.6167 - loss: 0.8967 - val_accuracy: 0.7320 - val_loss: 0.8655
Epoch 10/200
27/27 - 

In [80]:
class_report_train = classification_report(y_train, y_pred_train_classes)
class_report_test = classification_report(y_test, y_pred_test_classes)
print("Classification Report - Train:\n", class_report_train)
print("Classification Report - Test:\n", class_report_test)

Classification Report - Train:
               precision    recall  f1-score   support

           0       0.74      0.85      0.79       344
           1       0.75      0.57      0.64       283
           2       0.85      0.91      0.88       234

    accuracy                           0.77       861
   macro avg       0.78      0.77      0.77       861
weighted avg       0.77      0.77      0.77       861

Classification Report - Test:
               precision    recall  f1-score   support

           0       0.73      0.84      0.78        62
           1       0.72      0.62      0.67        53
           2       0.86      0.82      0.84        38

    accuracy                           0.76       153
   macro avg       0.77      0.76      0.76       153
weighted avg       0.76      0.76      0.76       153



In [81]:
model.save('maternal_health_risk_model.keras')

In [82]:
load_model = tf.keras.models.load_model('maternal_health_risk_model.keras')
y_new_pred = load_model.predict(x_test)
y_new_pred_classes = np.argmax(y_new_pred, axis=1)
report_new = classification_report(y_test, y_new_pred_classes)
print("Classification Report - Loaded Model Test:\n", report_new)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Classification Report - Loaded Model Test:
               precision    recall  f1-score   support

           0       0.73      0.84      0.78        62
           1       0.72      0.62      0.67        53
           2       0.86      0.82      0.84        38

    accuracy                           0.76       153
   macro avg       0.77      0.76      0.76       153
weighted avg       0.76      0.76      0.76       153



In [83]:
def permutation_importance(model, x_valid, y_valid, metric=accuracy_score):
    baseline = metric(y_valid, np.argmax(model.predict(x_valid), axis=1))
    importances = {}

    for i, col in enumerate(range(x_valid.shape[1])):
        x_temp = x_valid.copy()
        np.random.shuffle(x_temp[:, i])   # destroy information in this feature

        score = metric(y_valid, np.argmax(model.predict(x_temp), axis=1))
        importances[col] = baseline - score

    return importances

# Compute importances on test set
importances = permutation_importance(model, x_test, y_test)

# Sort by importance
sorted_importances = dict(sorted(importances.items(), key=lambda x: x[1], reverse=True))
sorted_importances

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


{2: 0.15032679738562094,
 4: 0.10457516339869277,
 6: 0.10457516339869277,
 3: 0.09150326797385622,
 7: 0.09150326797385622,
 8: 0.09150326797385622,
 9: 0.09150326797385622,
 5: 0.07843137254901955,
 0: 0.04575163398692805,
 1: 0.039215686274509776}